In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
Gemini_api_key = os.environ["GEMINI_API_KEY"]

In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain import hub

In [5]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import AstraDB

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
GoogleGeminiEmbeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=Gemini_api_key )

In [8]:
loader = PyPDFLoader("./Brown_Churchill_Complex_Variables_and_Ap.pdf")
pages = loader.load_and_split()

In [9]:
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1200, chunk_overlap = 400)
splits = text_splitter.split_documents(pages)

In [16]:
vstore = AstraDB(
    embedding=GoogleGeminiEmbeddings,
    collection_name="Complex_variables",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [35]:
from typing import List

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [97]:
class Key_Defintion(BaseModel):
    question: str = Field(description="A question asked")
    answer: str = Field(description = "answer of the question asked.")

parser = JsonOutputParser(pydantic_object=Key_Defintion)

key_word_quest_answer_prompt = PromptTemplate(
    template="Answer the user question briefly.\n{format_instructions}\n question : What is {query} in the context of Complex Analysis ?\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

key_word_chain = key_word_quest_answer_prompt | genai_llm | parser

In [98]:
parser = CommaSeparatedListOutputParser()
key_word_prompt = PromptTemplate(
    template="extract and list major Topics or sub topics from the given text which are related to the context of Complex Analysis. Example of keywords : Cauchy Reimann Equation, Taylor Series. Extract atmax 4 only. \n{format}.\n Text: {text}",
    input_variables=["text"],
    partial_variables={"format": parser.get_format_instructions()}
)
keyword_extract_chain = key_word_prompt | llm | parser

In [131]:
keyword_queries = []

In [102]:
def convert_dict_content(jsn: dict):
    return f"question: {jsn["question"]} \n answer: {jsn["answer"]}"

In [ ]:
print(len(splits))
for i,split in enumerate(splits[150:]):
    # extract keywords
    keywords = keyword_extract_chain.invoke({"text": split})
    print(f"{150+i}th split")
    # generate question and answer 
    for keyword in keywords:
        try:
            quest_answer = key_word_chain.invoke({"query": keyword})
            doc = convert_dict_content(quest_answer)
        except :
            print("Exception found Keyword: ",keyword )
            continue
        finally:
            keyword_queries.append(Document(page_content=doc, metadata= {"source": "Prerequisite", "page": 0}))

In [136]:
len(keyword_queries)

2293

In [137]:
vstore.add_documents(keyword_queries)

['17832e95013d45cb9c9e0acb9a6ac453',
 'eb1c402b819640f98c38d08f7349d324',
 '7306d47e82ec4489add2af367afb138d',
 'af0ef21f52a149519d95207e9abf4bd6',
 '693e3d2283f94c8f8f7725426d9bf7e6',
 'cd4182b95cd94465b9a27c9ca9aff4e5',
 'f23e4a40dfa043e0971223c7399600cd',
 'c34f21111c814079abc44f97b2ab8d63',
 '1f1e71f8ec84484bb50d1ab8a9bd8e5a',
 '92ac958352e1497fb178d83efbe36d2a',
 '2edd6517b5ff4af79eda97f245f0cbf6',
 'ff245b5421d64abc82faa8d8fdc9e0d4',
 'b63c767b08684e93a0dcb000192aa4b7',
 '8305be34b5d446c1a728e31cd543579f',
 '7dd08c58e648439d9b6fc16458df95e0',
 '56820749d3b64024a8610f5ad4496ab0',
 '8b2bb37e042c46ae90d05dbdc663deb2',
 '94072a91937f4ff1b3e063d657d4a494',
 '0091f08b762744c4bdbf91bbb8048b7c',
 '844b516fa4874dde8c7251be68b85b61',
 '66c6abde64994c6790ec89e145dfa30f',
 '28adf37fc5f8487d84af1a2812b9d00d',
 '5320bf4e05a34276949f88eab0ab727e',
 'c5aaad1eabb542be9336bc295974788e',
 'a0d3674568614b01b63e42e9c3f16b90',
 '04e0c2a0d67149eb8f8721d8ba888edf',
 'd140445d835043b9815e6d6f96983a28',
 

In [17]:
retriever = vstore.as_retriever()
template = """
context: {context}

question: {question}
"""
prompt = PromptTemplate.from_template(template)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.0, google_api_key=Gemini_api_key, )

In [25]:
genai_llm = GoogleGenerativeAI(model = "gemini-pro", temperature=0.0, google_api_key=Gemini_api_key)

In [22]:
def pages_adjustments(pages):
    return "\n\n".join(page.page_content for page in pages)

In [23]:
context_chain = retriever | pages_adjustments

In [26]:
main_chain = (
    {"context" : context_chain, "question": RunnablePassthrough()}
    |prompt
    | genai_llm
    | StrOutputParser()
)

In [142]:
print(main_chain.invoke("How to prove the Cauchy Reimann Equations?"))

The context does not mention anything about how to prove the Cauchy Reimann Equations, so I cannot extract the requested data from the provided context.


In [50]:
check_prompt = PromptTemplate.from_template("""
If the given statement is just a task then pick the keywords and make a questions of them to know them and then add the task also after a dot. 
statement: {statement}
""")
task_check_chain = (
    {"statement": RunnablePassthrough()}
    | check_prompt
    | genai_llm
    | StrOutputParser()
)

In [139]:
print(context_chain.invoke("How to prove Cauchy Reimann Equation?"))

question: What is Cauchy Reimann Equation in the context of Complex Analysis ? 
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are necessary and sufficient for a function to be holomorphic (analytic) in a domain.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ? 
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are necessary and sufficient for a function to be holomorphic (analytic) in a domain.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ? 
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are necessary and sufficient for a function to be holomorphic (analytic) in a domain.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ? 
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are necessary and sufficient for a function

In [140]:
print(context_chain.invoke("What is Cauchy Reimann Equation? Also Proof the equations."))

question: What is Cauchy Reimann Equation in the context of Complex Analysis ?  
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are satisfied by the real and imaginary parts of a holomorphic function.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ?  
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are satisfied by the real and imaginary parts of a holomorphic function.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ?  
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are satisfied by the real and imaginary parts of a holomorphic function.

question: What is Cauchy Reimann Equation in the context of Complex Analysis ?  
 answer: The Cauchy-Riemann equations are a system of two partial differential equations that are satisfied by the real and imaginary parts of a holomorphic function.
